In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import copy
import pandas as pd
from torch_geometric.datasets import TUDataset, ZINC
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool, global_max_pool, global_add_pool
from torch_geometric.nn import BatchNorm, LayerNorm
from codecarbon import EmissionsTracker

def set_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GNNSearchSpace:
    def __init__(self):
        self.conv_types = ['GCN', 'GAT', 'SAGE']
        self.pooling_types = ['mean', 'max', 'add']
        self.activation_types = ['relu', 'elu']
        self.norm_types = ['batch', 'layer', 'none']
        self.hidden_dims = [32, 64, 128]
        self.num_layers = [2, 3, 4]
        self.dropout_rates = [0.0, 0.2, 0.5]
    def sample_architecture(self):
        return {
            'conv_type': random.choice(self.conv_types),
            'pooling_type': random.choice(self.pooling_types),
            'activation': random.choice(self.activation_types),
            'norm_type': random.choice(self.norm_types),
            'hidden_dim': random.choice(self.hidden_dims),
            'num_layers': random.choice(self.num_layers),
            'dropout': random.choice(self.dropout_rates)
        }

class GNNModel(nn.Module):
    def __init__(self, config, input_dim, output_dim):
        super(GNNModel, self).__init__()
        self.config = config
        self.num_layers = config['num_layers']
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        in_dim = input_dim
        hidden_dim = config['hidden_dim']
        for i in range(self.num_layers):
            out_dim = hidden_dim
            if config['conv_type'] == 'GCN':
                conv = GCNConv(in_dim, out_dim)
            elif config['conv_type'] == 'GAT':
                conv = GATConv(in_dim, out_dim, heads=1)
            elif config['conv_type'] == 'SAGE':
                conv = SAGEConv(in_dim, out_dim)
            self.convs.append(conv)
            if i < self.num_layers - 1:
                if config['norm_type'] == 'batch':
                    norm = BatchNorm(out_dim)
                elif config['norm_type'] == 'layer':
                    norm = LayerNorm(out_dim)
                else:
                    norm = nn.Identity()
                self.norms.append(norm)
            in_dim = out_dim
        if config['pooling_type'] == 'mean':
            self.pool = global_mean_pool
        elif config['pooling_type'] == 'max':
            self.pool = global_max_pool
        elif config['pooling_type'] == 'add':
            self.pool = global_add_pool
        self.classifier = nn.Linear(hidden_dim, output_dim)
    def forward(self, x, edge_index, batch=None):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            if i < self.num_layers - 1:
                x = self.norms[i](x)
                if self.config['activation'] == 'relu':
                    x = F.relu(x)
                elif self.config['activation'] == 'elu':
                    x = F.elu(x)
                x = F.dropout(x, p=self.config['dropout'], training=self.training)
        if batch is not None:
            x = self.pool(x, batch)
            x = self.classifier(x)
        return x

class CarbonAwareGNNNAS:
    def __init__(self, dataset_name='MUTAG', population_size=12, generations=5, seed=42):
        set_seed(seed)
        self.dataset_name = dataset_name
        self.population_size = population_size
        self.generations = generations
        self.search_space = GNNSearchSpace()
        self.load_dataset()
        self.results = []
        self.best_architecture = None
        self.best_score = 0.0

    def load_dataset(self):
        if self.dataset_name in ['MUTAG', 'PROTEINS', 'ENZYMES']:
            dataset = TUDataset(root=f'data/{self.dataset_name}', name=self.dataset_name)
        elif self.dataset_name == 'ZINC':
            dataset = ZINC(root='data/ZINC', subset=True, split='train')
        else:
            raise ValueError(f"Unsupported dataset: {self.dataset_name}")
        dataset = dataset.shuffle()
        n = len(dataset)
        train_size = int(0.8 * n)
        val_size = int(0.1 * n)
        self.train_dataset = dataset[:train_size]
        self.val_dataset = dataset[train_size:train_size+val_size]
        self.test_dataset = dataset[train_size+val_size:]
        self.train_loader = DataLoader(self.train_dataset, batch_size=64, shuffle=True)
        self.val_loader = DataLoader(self.val_dataset, batch_size=64, shuffle=False)
        self.test_loader = DataLoader(self.test_dataset, batch_size=64, shuffle=False)
        self.input_dim = dataset.num_node_features
        self.output_dim = dataset.num_classes
        print(f"Dataset: {self.dataset_name}")

    def calculate_block_reuse(self, arch1, arch2):
        reuse_score = 0.0
        total = len(arch1)
        for k in arch1.keys():
            if arch1[k] == arch2[k]:
                reuse_score += 1
        return reuse_score / total

    def estimate_proxy_carbon(self, config):
        layer_weights = {'GCN': 1.0, 'GAT': 1.5, 'SAGE': 1.2}
        norm_weights = {'batch': 0.2, 'layer': 0.1, 'none': 0.0}
        return (layer_weights[config['conv_type']] +
                config['hidden_dim'] / 128.0 +
                config['num_layers'] / 4.0 +
                norm_weights[config['norm_type']])

    # Optimized proxy training with early stopping
    def proxy_train_and_eval(self, config, max_epochs, acc_threshold=0.38, check_every=5, min_epochs=15):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = GNNModel(config, self.input_dim, self.output_dim).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        criterion = nn.CrossEntropyLoss()
        tracker = EmissionsTracker(project_name=f"gnn_nas_proxy_{self.dataset_name}", measure_power_secs=15)
        tracker.start()
        best_acc = 0.0
        for epoch in range(max_epochs):
            model.train()
            for batch in self.train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = criterion(out, batch.y)
                loss.backward()
                optimizer.step()
            if (epoch+1) % check_every == 0 or epoch == max_epochs-1:
                acc = self.simple_eval(model, device)
                best_acc = max(best_acc, acc)
                # Early stop if not enough progress after min_epochs
                if epoch+1 >= min_epochs and best_acc < acc_threshold:
                    break
                if best_acc >= acc_threshold:
                    break
        emissions = tracker.stop()
        return {'accuracy': best_acc,
                'carbon': emissions if emissions else self.estimate_proxy_carbon(config) * 0.01,
                'model': model}

    def simple_eval(self, model, device):
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in self.val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item()
                total += batch.y.size(0)
        return correct/total if total > 0 else 0.0

    def train_and_evaluate(self, config, epochs=20):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = GNNModel(config, self.input_dim, self.output_dim).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        criterion = nn.CrossEntropyLoss()
        tracker = EmissionsTracker(project_name=f"gnn_nas_{self.dataset_name}", measure_power_secs=15)
        tracker.start()
        model.train()
        for epoch in range(epochs):
            for batch in self.train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = criterion(out, batch.y)
                loss.backward()
                optimizer.step()
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in self.val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item()
                total += batch.y.size(0)
        accuracy = correct / total if total > 0 else 0.0
        emissions = tracker.stop()
        return {'accuracy': accuracy,
                'carbon': emissions if emissions else self.estimate_proxy_carbon(config) * 0.01,
                'model': model}

    def mutate_architecture(self, base_config):
        if base_config is None:
            return self.search_space.sample_architecture()
        new_config = copy.deepcopy(base_config)
        all_options = {
            'conv_type': self.search_space.conv_types,
            'pooling_type': self.search_space.pooling_types,
            'activation': self.search_space.activation_types,
            'norm_type': self.search_space.norm_types,
            'hidden_dim': self.search_space.hidden_dims,
            'num_layers': self.search_space.num_layers,
            'dropout': self.search_space.dropout_rates,
        }
        component = random.choice(list(all_options.keys()))
        cur_val = new_config[component]
        possible_vals = [v for v in all_options[component] if v != cur_val]
        new_config[component] = random.choice(possible_vals) if possible_vals else cur_val
        return new_config

    def run_search(self):
        print("Starting Carbon-Aware GNN NAS...")
        baseline_config = {
            'conv_type': 'GCN', 'pooling_type': 'mean', 'activation': 'relu',
            'norm_type': 'batch', 'hidden_dim': 64, 'num_layers': 2, 'dropout': 0.2
        }
        print("Training baseline...")
        baseline_result = self.train_and_evaluate(baseline_config, epochs=40)
        self.baseline_config = baseline_config
        self.baseline_result = baseline_result
        print(f"Baseline - Accuracy: {baseline_result['accuracy']:.4f}, Carbon: {baseline_result['carbon']:.6f}")

        for generation in range(self.generations):
            print(f"\nGeneration {generation + 1}/{self.generations}")
            if generation == 0:
                print("  Sampling initial viable population (min carbon, early stopping)...")
                population = []
                attempts = 0
                max_attempts = 500 # Avoid infinite loops
                while len(population) < self.population_size and attempts < max_attempts:
                    config = self.search_space.sample_architecture()
                    result = self.proxy_train_and_eval(config, max_epochs=50, acc_threshold=0.38)
                    if result['accuracy'] >= 0.38:
                        population.append({
                            'config': config,
                            'proxy_acc': result['accuracy'],
                            'carbon': result['carbon'],
                            'model': result['model']
                        })
                        print(f"    Found candidate {len(population)} with Proxy Acc={result['accuracy']:.4f}, Carbon={result['carbon']:.6f}")
                    attempts += 1
                if len(population) < self.population_size:
                    print(f"  Warning: Only {len(population)} viable candidates found after {attempts} attempts!")
                # Full eval for survivors
                final_population = []
                for idx, candidate in enumerate(population, 1):
                    full_result = self.train_and_evaluate(candidate['config'], epochs=30)
                    reuse_score = self.calculate_block_reuse(candidate['config'], self.baseline_config)
                    final_population.append({
                        'config': candidate['config'],
                        'accuracy': full_result['accuracy'],
                        'carbon': full_result['carbon'],
                        'reuse': reuse_score,
                        'model': full_result['model']
                    })
                    print(f"    Final candidate {idx}: Acc={full_result['accuracy']:.4f}, Carbon={full_result['carbon']:.6f}, Reuse={reuse_score:.3f}")
                population = final_population
            else:
                current_population_size = self.population_size
                population = []
                attempts = 0
                max_attempts = current_population_size * 20
                while len(population) < current_population_size and attempts < max_attempts:
                    config = self.mutate_architecture(self.best_architecture)
                    result = self.proxy_train_and_eval(config, max_epochs=20, acc_threshold=0.38)
                    if result['accuracy'] >= 0.38:
                        full_result = self.train_and_evaluate(config, epochs=30)
                        reuse_score = self.calculate_block_reuse(config, self.baseline_config)
                        population.append({
                            'config': config,
                            'accuracy': full_result['accuracy'],
                            'carbon': full_result['carbon'],
                            'reuse': reuse_score,
                            'model': full_result['model']
                        })
                        print(f"  Candidate {len(population)}: Proxy Acc={result['accuracy']:.4f}, "
                              f"Acc={full_result['accuracy']:.4f}, Carbon={full_result['carbon']:.6f}, "
                              f"Reuse={reuse_score:.3f}")
                    attempts += 1
                if len(population) < current_population_size:
                    print(f"  Warning: Only {len(population)} viable candidates found after {attempts} attempts.")

            # Score and select best as before
            if population:
                for candidate in population:
                    score = (candidate['accuracy'] * 0.5 +
                             (1 - min(candidate['carbon'], 0.1) / 0.1) * 0.3 +
                             candidate['reuse'] * 0.2)
                    candidate['score'] = score
                population.sort(key=lambda x: x['score'], reverse=True)
                best_candidate = population[0]
                if best_candidate['score'] > self.best_score or self.best_architecture is None:
                    self.best_score = best_candidate['score']
                    self.best_architecture = best_candidate['config']
                    self.best_model = best_candidate['model']
                print(f"  Best candidate: Score={best_candidate['score']:.4f}")
                self.results.append(best_candidate)
            else:
                print(f"  Warning: No candidates survived in generation {generation + 1}. Restarting from random.")
                self.best_architecture = self.search_space.sample_architecture()

        print("\nRetraining best architecture on full data...")
        final_result = self.train_and_evaluate(self.best_architecture, epochs=80)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.best_model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in self.test_loader:
                batch = batch.to(device)
                out = self.best_model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item()
                total += batch.y.size(0)
        test_accuracy = correct / total if total > 0 else 0.0
        print("\nFinal Results:")
        print(f"Best Architecture: {self.best_architecture}")
        print(f"Final Test Accuracy: {test_accuracy:.4f}")
        print(f"Final Carbon Footprint: {final_result['carbon']:.6f} kg CO2")
        print(f"Block Reuse Score: {self.calculate_block_reuse(self.best_architecture, self.baseline_config):.3f}")
        results_df = pd.DataFrame([{
            'dataset': self.dataset_name,
            'best_config': str(self.best_architecture),
            'test_accuracy': test_accuracy,
            'carbon_footprint': final_result['carbon'],
            'reuse_score': self.calculate_block_reuse(self.best_architecture, self.baseline_config)
        }])
        results_df.to_csv(f'{self.dataset_name.lower()}_gnn_nas_results.csv', index=False)

if __name__ == "__main__":
    nas = CarbonAwareGNNNAS(dataset_name='MUTAG', population_size=12, generations=5)
    nas.run_search()


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
[codecarbon WARNING @ 18:01:05] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 18:01:05] [setup] RAM Tracking...
[codecarbon INFO @ 18:01:05] [setup] CPU Tracking...


Dataset: MUTAG
Starting Carbon-Aware GNN NAS...
Training baseline...


[codecarbon WARNING @ 18:01:07] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 18:01:07] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 18:01:07] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 18:01:07] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:01:07] [setup] GPU Tracking...
[codecarbon INFO @ 18:01:07] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:01:07] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 18:01:07] >>> Tracker's metadata:
[codecarbon INFO @ 18:01:07]   Platform sys

Baseline - Accuracy: 0.5556, Carbon: 0.000013

Generation 1/5
  Sampling initial viable population (min carbon, early stopping)...


[codecarbon WARNING @ 18:01:12] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 18:01:12] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 18:01:12] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 18:01:12] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:01:12] [setup] GPU Tracking...
[codecarbon INFO @ 18:01:12] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:01:12] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 18:01:12] >>> Tracker's metadata:
[codecarbon INFO @ 18:01:12]   Platform sys

...

[codecarbon WARNING @ 18:10:29] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 18:10:29] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 18:10:29] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 18:10:29] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:10:29] [setup] GPU Tracking...
[codecarbon INFO @ 18:10:29] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:10:29] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 18:10:29] >>> Tracker's metadata:
[codecarbon INFO @ 18:10:29]   Platform sys

  Candidate 12: Proxy Acc=0.6111, Acc=0.5000, Carbon=0.000007, Reuse=0.571
  Best candidate: Score=0.7770

Retraining best architecture on full data...


[codecarbon WARNING @ 18:10:39] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 18:10:39] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 18:10:39] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 18:10:39] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:10:39] [setup] GPU Tracking...
[codecarbon INFO @ 18:10:39] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:10:39] Energy consumed for RAM : 0.042810 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 18:10:39] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:10:39] Energy consumed for All CPU : 0.090977 kWh
[codecarbon INFO @ 18:10:39] The below tracking methods have been set up:
      


Final Results:
Best Architecture: {'conv_type': 'GCN', 'pooling_type': 'add', 'activation': 'relu', 'norm_type': 'batch', 'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.2}
Final Test Accuracy: 0.7500
Final Carbon Footprint: 0.000019 kg CO2
Block Reuse Score: 0.714


[codecarbon INFO @ 18:10:46] Energy consumed for RAM : 0.069045 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 18:10:46] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:10:46] Energy consumed for All CPU : 0.146732 kWh
[codecarbon INFO @ 18:10:46] Energy consumed for all GPUs : 0.150726 kWh. Total GPU Power : 42.7682034611673 W
[codecarbon INFO @ 18:10:46] 0.366503 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:10:54] Energy consumed for RAM : 0.042893 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 18:10:54] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:10:54] Energy consumed for All CPU : 0.091154 kWh
[codecarbon INFO @ 18:10:54] Energy consumed for all GPUs : 0.093535 kWh. Total GPU Power : 42.917487596316775 W
[codecarbon INFO @ 18:10:54] 0.227583 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:11:01] Energy consumed for RAM : 0.069129 kWh. RAM Power : 2

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import copy
import pandas as pd
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool, global_max_pool, global_add_pool
from torch_geometric.nn import BatchNorm, LayerNorm
from codecarbon import EmissionsTracker

def set_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GNNSearchSpace:
    def __init__(self):
        self.conv_types = ['GCN', 'GAT', 'SAGE']
        self.pooling_types = ['mean', 'max', 'add']
        self.activation_types = ['relu', 'elu']
        self.norm_types = ['batch', 'layer', 'none']
        self.hidden_dims = [64, 128, 256]           # wider for medium dataset
        self.num_layers = [2, 3, 4]
        self.dropout_rates = [0.2, 0.5]             # higher dropout for PROTEINS
    def sample_architecture(self):
        return {
            'conv_type': random.choice(self.conv_types),
            'pooling_type': random.choice(self.pooling_types),
            'activation': random.choice(self.activation_types),
            'norm_type': random.choice(self.norm_types),
            'hidden_dim': random.choice(self.hidden_dims),
            'num_layers': random.choice(self.num_layers),
            'dropout': random.choice(self.dropout_rates)
        }

class GNNModel(nn.Module):
    def __init__(self, config, input_dim, output_dim):
        super(GNNModel, self).__init__()
        self.config = config
        self.num_layers = config['num_layers']
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        in_dim = input_dim
        hidden_dim = config['hidden_dim']
        for i in range(self.num_layers):
            out_dim = hidden_dim
            if config['conv_type'] == 'GCN':
                conv = GCNConv(in_dim, out_dim)
            elif config['conv_type'] == 'GAT':
                conv = GATConv(in_dim, out_dim, heads=1)
            elif config['conv_type'] == 'SAGE':
                conv = SAGEConv(in_dim, out_dim)
            self.convs.append(conv)
            if i < self.num_layers - 1:
                if config['norm_type'] == 'batch':
                    norm = BatchNorm(out_dim)
                elif config['norm_type'] == 'layer':
                    norm = LayerNorm(out_dim)
                else:
                    norm = nn.Identity()
                self.norms.append(norm)
            in_dim = out_dim
        if config['pooling_type'] == 'mean':
            self.pool = global_mean_pool
        elif config['pooling_type'] == 'max':
            self.pool = global_max_pool
        elif config['pooling_type'] == 'add':
            self.pool = global_add_pool
        self.classifier = nn.Linear(hidden_dim, output_dim)
    def forward(self, x, edge_index, batch=None):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            if i < self.num_layers - 1:
                x = self.norms[i](x)
                if self.config['activation'] == 'relu':
                    x = F.relu(x)
                elif self.config['activation'] == 'elu':
                    x = F.elu(x)
                x = F.dropout(x, p=self.config['dropout'], training=self.training)
        if batch is not None:
            x = self.pool(x, batch)
            x = self.classifier(x)
        return x

class CarbonAwareGNNNAS:
    def __init__(self, dataset_name='PROTEINS', population_size=12, generations=5, seed=42):
        set_seed(seed)
        self.dataset_name = dataset_name
        self.population_size = population_size
        self.generations = generations
        self.search_space = GNNSearchSpace()
        self.output_file = f"emissions_{self.dataset_name.lower()}.csv"  # Set here for easy reuse
        self.load_dataset()
        self.results = []
        self.best_architecture = None
        self.best_score = 0.0

    def load_dataset(self):
        dataset = TUDataset(root=f'data/{self.dataset_name}', name=self.dataset_name)
        dataset = dataset.shuffle()
        n = len(dataset)
        train_size = int(0.8 * n)
        val_size = int(0.1 * n)
        self.train_dataset = dataset[:train_size]
        self.val_dataset = dataset[train_size:train_size+val_size]
        self.test_dataset = dataset[train_size+val_size:]
        self.train_loader = DataLoader(self.train_dataset, batch_size=64, shuffle=True)
        self.val_loader = DataLoader(self.val_dataset, batch_size=64, shuffle=False)
        self.test_loader = DataLoader(self.test_dataset, batch_size=64, shuffle=False)
        self.input_dim = dataset.num_node_features
        self.output_dim = dataset.num_classes
        print(f"Dataset: {self.dataset_name}")

    def calculate_block_reuse(self, arch1, arch2):
        reuse_score = 0.0
        total = len(arch1)
        for k in arch1.keys():
            if arch1[k] == arch2[k]:
                reuse_score += 1
        return reuse_score / total

    def estimate_proxy_carbon(self, config):
        layer_weights = {'GCN': 1.0, 'GAT': 1.5, 'SAGE': 1.2}
        norm_weights = {'batch': 0.2, 'layer': 0.1, 'none': 0.0}
        return (layer_weights[config['conv_type']] +
                config['hidden_dim'] / 128.0 +
                config['num_layers'] / 4.0 +
                norm_weights[config['norm_type']])

    # Add output_file to every EmissionsTracker usage
    def proxy_train_and_eval(self, config, max_epochs, acc_threshold=0.5, check_every=5, min_epochs=15):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = GNNModel(config, self.input_dim, self.output_dim).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        criterion = nn.CrossEntropyLoss()
        tracker = EmissionsTracker(
            project_name=f"gnn_nas_proxy_{self.dataset_name}",
            measure_power_secs=15,
            output_file=self.output_file
        )
        tracker.start()
        best_acc = 0.0
        for epoch in range(max_epochs):
            model.train()
            for batch in self.train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = criterion(out, batch.y)
                loss.backward()
                optimizer.step()
            if (epoch+1) % check_every == 0 or epoch == max_epochs-1:
                acc = self.simple_eval(model, device)
                best_acc = max(best_acc, acc)
                if epoch+1 >= min_epochs and best_acc < acc_threshold:
                    break
                if best_acc >= acc_threshold:
                    break
        emissions = tracker.stop()
        return {'accuracy': best_acc,
                'carbon': emissions if emissions else self.estimate_proxy_carbon(config) * 0.01,
                'model': model}

    def simple_eval(self, model, device):
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in self.val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item()
                total += batch.y.size(0)
        return correct/total if total > 0 else 0.0

    def train_and_evaluate(self, config, epochs=40):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = GNNModel(config, self.input_dim, self.output_dim).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        criterion = nn.CrossEntropyLoss()
        tracker = EmissionsTracker(
            project_name=f"gnn_nas_{self.dataset_name}",
            measure_power_secs=15,
            output_file=self.output_file
        )
        tracker.start()
        model.train()
        for epoch in range(epochs):
            for batch in self.train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = criterion(out, batch.y)
                loss.backward()
                optimizer.step()
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in self.val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item()
                total += batch.y.size(0)
        accuracy = correct / total if total > 0 else 0.0
        emissions = tracker.stop()
        return {'accuracy': accuracy,
                'carbon': emissions if emissions else self.estimate_proxy_carbon(config) * 0.01,
                'model': model}

    def mutate_architecture(self, base_config):
        if base_config is None:
            return self.search_space.sample_architecture()
        new_config = copy.deepcopy(base_config)
        all_options = {
            'conv_type': self.search_space.conv_types,
            'pooling_type': self.search_space.pooling_types,
            'activation': self.search_space.activation_types,
            'norm_type': self.search_space.norm_types,
            'hidden_dim': self.search_space.hidden_dims,
            'num_layers': self.search_space.num_layers,
            'dropout': self.search_space.dropout_rates,
        }
        component = random.choice(list(all_options.keys()))
        cur_val = new_config[component]
        possible_vals = [v for v in all_options[component] if v != cur_val]
        new_config[component] = random.choice(possible_vals) if possible_vals else cur_val
        return new_config

    def run_search(self):
        print("Starting Carbon-Aware GNN NAS...")
        baseline_config = {
            'conv_type': 'GCN', 'pooling_type': 'mean', 'activation': 'relu',
            'norm_type': 'batch', 'hidden_dim': 64, 'num_layers': 2, 'dropout': 0.2
        }
        print("Training baseline...")
        baseline_result = self.train_and_evaluate(baseline_config, epochs=40)
        self.baseline_config = baseline_config
        self.baseline_result = baseline_result
        print(f"Baseline - Accuracy: {baseline_result['accuracy']:.4f}, Carbon: {baseline_result['carbon']:.6f}")

        max_proxy_epochs_first_gen = 60
        max_proxy_epochs_others = 24
        proxy_acc_threshold = 0.5
        full_training_epochs = 40

        for generation in range(self.generations):
            print(f"\nGeneration {generation + 1}/{self.generations}")
            if generation == 0:
                print("  Sampling initial viable population (with proxy early stopping)...")
                population = []
                attempts = 0
                max_attempts = 700
                while len(population) < self.population_size and attempts < max_attempts:
                    config = self.search_space.sample_architecture()
                    result = self.proxy_train_and_eval(config, max_epochs=max_proxy_epochs_first_gen, acc_threshold=proxy_acc_threshold)
                    if result['accuracy'] >= proxy_acc_threshold:
                        population.append({
                            'config': config,
                            'proxy_acc': result['accuracy'],
                            'carbon': result['carbon'],
                            'model': result['model']
                        })
                        print(f"    Found candidate {len(population)} with Proxy Acc={result['accuracy']:.4f}, Carbon={result['carbon']:.6f}")
                    attempts += 1
                if len(population) < self.population_size:
                    print(f"  Warning: Only {len(population)} viable candidates found after {attempts} attempts!")
                final_population = []
                for idx, candidate in enumerate(population, 1):
                    full_result = self.train_and_evaluate(candidate['config'], epochs=full_training_epochs)
                    reuse_score = self.calculate_block_reuse(candidate['config'], self.baseline_config)
                    final_population.append({
                        'config': candidate['config'],
                        'accuracy': full_result['accuracy'],
                        'carbon': full_result['carbon'],
                        'reuse': reuse_score,
                        'model': full_result['model']
                    })
                    print(f"    Final candidate {idx}: Acc={full_result['accuracy']:.4f}, Carbon={full_result['carbon']:.6f}, Reuse={reuse_score:.3f}")
                population = final_population
            else:
                population = []
                attempts = 0
                max_attempts = self.population_size * 20
                while len(population) < self.population_size and attempts < max_attempts:
                    config = self.mutate_architecture(self.best_architecture)
                    result = self.proxy_train_and_eval(config, max_epochs=max_proxy_epochs_others, acc_threshold=proxy_acc_threshold)
                    if result['accuracy'] >= proxy_acc_threshold:
                        full_result = self.train_and_evaluate(config, epochs=full_training_epochs)
                        reuse_score = self.calculate_block_reuse(config, self.baseline_config)
                        population.append({
                            'config': config,
                            'accuracy': full_result['accuracy'],
                            'carbon': full_result['carbon'],
                            'reuse': reuse_score,
                            'model': full_result['model']
                        })
                        print(f"  Candidate {len(population)}: Proxy Acc={result['accuracy']:.4f}, "
                              f"Acc={full_result['accuracy']:.4f}, Carbon={full_result['carbon']:.6f}, "
                              f"Reuse={reuse_score:.3f}")
                    attempts += 1
                if len(population) < self.population_size:
                    print(f"  Warning: Only {len(population)} viable candidates found after {attempts} attempts.")

            if population:
                for candidate in population:
                    score = (candidate['accuracy'] * 0.5 +
                             (1 - min(candidate['carbon'], 0.1) / 0.1) * 0.3 +
                             candidate['reuse'] * 0.2)
                    candidate['score'] = score
                population.sort(key=lambda x: x['score'], reverse=True)
                best_candidate = population[0]
                if best_candidate['score'] > self.best_score or self.best_architecture is None:
                    self.best_score = best_candidate['score']
                    self.best_architecture = best_candidate['config']
                    self.best_model = best_candidate['model']
                print(f"  Best candidate: Score={best_candidate['score']:.4f}")
                self.results.append(best_candidate)
            else:
                print(f"  Warning: No candidates survived in generation {generation + 1}. Restarting from random.")
                self.best_architecture = self.search_space.sample_architecture()

        print("\nRetraining best architecture on full data...")
        final_result = self.train_and_evaluate(self.best_architecture, epochs=80)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.best_model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in self.test_loader:
                batch = batch.to(device)
                out = self.best_model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item()
                total += batch.y.size(0)
        test_accuracy = correct / total if total > 0 else 0.0
        print("\nFinal Results:")
        print(f"Best Architecture: {self.best_architecture}")
        print(f"Final Test Accuracy: {test_accuracy:.4f}")
        print(f"Final Carbon Footprint: {final_result['carbon']:.6f} kg CO2")
        print(f"Block Reuse Score: {self.calculate_block_reuse(self.best_architecture, self.baseline_config):.3f}")
        results_df = pd.DataFrame([{
            'dataset': self.dataset_name,
            'best_config': str(self.best_architecture),
            'test_accuracy': test_accuracy,
            'carbon_footprint': final_result['carbon'],
            'reuse_score': self.calculate_block_reuse(self.best_architecture, self.baseline_config)
        }])
        results_df.to_csv(f'{self.dataset_name.lower()}_gnn_nas_results.csv', index=False)

if __name__ == "__main__":
    nas = CarbonAwareGNNNAS(dataset_name='PROTEINS', population_size=12, generations=5)
    nas.run_search()


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
[codecarbon WARNING @ 19:30:53] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 19:30:53] [setup] RAM Tracking...
[codecarbon INFO @ 19:30:53] [setup] CPU Tracking...


Dataset: PROTEINS
Starting Carbon-Aware GNN NAS...
Training baseline...


[codecarbon WARNING @ 19:30:54] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 19:30:54] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 19:30:54] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 19:30:54] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 19:30:54] [setup] GPU Tracking...
[codecarbon INFO @ 19:30:54] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:30:54] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 19:30:54] >>> Tracker's metadata:
[codecarbon INFO @ 19:30:54]   Platform sys

Baseline - Accuracy: 0.6757, Carbon: 0.000054

Generation 1/5
  Sampling initial viable population (with proxy early stopping)...


[codecarbon INFO @ 19:31:01] Energy consumed for RAM : 0.095781 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:31:01] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 19:31:01] Energy consumed for All CPU : 0.203548 kWh
[codecarbon INFO @ 19:31:01] Energy consumed for all GPUs : 0.209630 kWh. Total GPU Power : 44.141971240213365 W
[codecarbon INFO @ 19:31:01] 0.508959 kWh of electricity used since the beginning.
[codecarbon WARNING @ 19:31:02] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 19:31:02] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 19:31:02] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 19:31:02] No CPU tracking mode found. Falling back on CPU constant mod

...

[codecarbon WARNING @ 19:44:14] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 19:44:14] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 19:44:14] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 19:44:14] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 19:44:14] [setup] GPU Tracking...
[codecarbon INFO @ 19:44:14] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:44:14] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 19:44:14] >>> Tracker's metadata:
[codecarbon INFO @ 19:44:14]   Platform sys

  Candidate 12: Proxy Acc=0.5766, Acc=0.7117, Carbon=0.000051, Reuse=0.714
  Best candidate: Score=0.8496

Retraining best architecture on full data...


[codecarbon WARNING @ 19:44:27] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 19:44:27] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 19:44:27] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 19:44:27] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 19:44:27] [setup] GPU Tracking...
[codecarbon INFO @ 19:44:27] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:44:27] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 19:44:27] >>> Tracker's metadata:
[codecarbon INFO @ 19:44:27]   Platform sys


Final Results:
Best Architecture: {'conv_type': 'GCN', 'pooling_type': 'add', 'activation': 'relu', 'norm_type': 'batch', 'hidden_dim': 64, 'num_layers': 2, 'dropout': 0.2}
Final Test Accuracy: 0.7589
Final Carbon Footprint: 0.000103 kg CO2
Block Reuse Score: 0.857


[codecarbon INFO @ 19:44:39] Energy consumed for RAM : 0.074126 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:44:39] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 19:44:39] Energy consumed for All CPU : 0.157529 kWh
[codecarbon INFO @ 19:44:39] Energy consumed for all GPUs : 0.162863 kWh. Total GPU Power : 45.337589012817865 W
[codecarbon INFO @ 19:44:39] 0.394518 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:44:46] Energy consumed for RAM : 0.100361 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:44:46] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 19:44:46] Energy consumed for All CPU : 0.213283 kWh
[codecarbon INFO @ 19:44:46] Energy consumed for all GPUs : 0.220236 kWh. Total GPU Power : 45.28261480626172 W
[codecarbon INFO @ 19:44:46] 0.533880 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:44:54] Energy consumed for RAM : 0.074209 kWh. RAM Power : 

In [3]:
#NCl1 DATASET
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
!pip install torch-geometric
!pip install codecarbon
!pip install pandas
!pip install scikit-learn
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import copy
import pandas as pd
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool, global_max_pool, global_add_pool
from torch_geometric.nn import BatchNorm, LayerNorm
from codecarbon import EmissionsTracker

def set_seed(seed=42):
    """Sets a random seed for reproducibility."""
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GNNSearchSpace:
    """Defines the architectural search space for GNNs."""
    def __init__(self):
        self.conv_types = ['GCN', 'GAT', 'SAGE']
        self.pooling_types = ['mean', 'max', 'add']
        self.activation_types = ['relu', 'elu']
        self.norm_types = ['batch', 'layer', 'none']
        self.hidden_dims = [64, 128, 256]
        self.num_layers = [2, 3, 4, 5]
        self.dropout_rates = [0.2, 0.5]

    def sample_architecture(self):
        """Randomly samples an architecture from the search space."""
        return {
            'conv_type': random.choice(self.conv_types),
            'pooling_type': random.choice(self.pooling_types),
            'activation': random.choice(self.activation_types),
            'norm_type': random.choice(self.norm_types),
            'hidden_dim': random.choice(self.hidden_dims),
            'num_layers': random.choice(self.num_layers),
            'dropout': random.choice(self.dropout_rates)
        }

class GNNModel(nn.Module):
    """A flexible GNN model built from a configuration dictionary."""
    def __init__(self, config, input_dim, output_dim):
        super(GNNModel, self).__init__()
        self.config = config
        self.num_layers = config['num_layers']
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        in_dim = input_dim
        hidden_dim = config['hidden_dim']
        for i in range(self.num_layers):
            out_dim = hidden_dim
            if config['conv_type'] == 'GCN':
                conv = GCNConv(in_dim, out_dim)
            elif config['conv_type'] == 'GAT':
                conv = GATConv(in_dim, out_dim, heads=1)
            elif config['conv_type'] == 'SAGE':
                conv = SAGEConv(in_dim, out_dim)
            self.convs.append(conv)
            if i < self.num_layers - 1:
                if config['norm_type'] == 'batch':
                    norm = BatchNorm(out_dim)
                elif config['norm_type'] == 'layer':
                    norm = LayerNorm(out_dim)
                else: # 'none'
                    norm = nn.Identity()
                self.norms.append(norm)
            in_dim = out_dim
        if config['pooling_type'] == 'mean':
            self.pool = global_mean_pool
        elif config['pooling_type'] == 'max':
            self.pool = global_max_pool
        elif config['pooling_type'] == 'add':
            self.pool = global_add_pool
        self.classifier = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, batch=None):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            if i < self.num_layers - 1:
                x = self.norms[i](x)
                if self.config['activation'] == 'relu':
                    x = F.relu(x)
                elif self.config['activation'] == 'elu':
                    x = F.elu(x)
                x = F.dropout(x, p=self.config['dropout'], training=self.training)
        if batch is not None:
            x = self.pool(x, batch)
        x = self.classifier(x)
        return x

class CarbonAwareGNNNAS:
    """The main class for running the Carbon-Aware NAS process."""
    def __init__(self, dataset_name='NCI1', population_size=12, generations=5, seed=42):
        set_seed(seed)
        self.dataset_name = dataset_name
        self.population_size = population_size
        self.generations = generations
        self.search_space = GNNSearchSpace()
        self.output_file = f"{self.dataset_name}_emissions.csv"
        self.load_dataset()
        self.results = []
        self.best_architecture = None
        self.best_score = 0.0
        self.best_model = None

    def load_dataset(self):
        """Loads and splits the specified TUDataset."""
        dataset = TUDataset(root=f'data/{self.dataset_name}', name=self.dataset_name)
        dataset = dataset.shuffle()
        n = len(dataset)
        train_size = int(0.8 * n)
        val_size = int(0.1 * n)
        self.train_dataset = dataset[:train_size]
        self.val_dataset = dataset[train_size:train_size+val_size]
        self.test_dataset = dataset[train_size+val_size:]
        self.train_loader = DataLoader(self.train_dataset, batch_size=128, shuffle=True)
        self.val_loader = DataLoader(self.val_dataset, batch_size=128, shuffle=False)
        self.test_loader = DataLoader(self.test_dataset, batch_size=128, shuffle=False)
        self.input_dim = dataset.num_node_features
        self.output_dim = dataset.num_classes
        print(f"Dataset: {self.dataset_name} | Graphs: {len(dataset)} | Classes: {self.output_dim} | Features: {self.input_dim}")

    def calculate_block_reuse(self, arch1, arch2):
        """Calculates the proportion of shared components between two architectures."""
        reuse_score = 0.0
        for k in arch1.keys():
            if arch1[k] == arch2[k]:
                reuse_score += 1
        return reuse_score / len(arch1)

    def estimate_proxy_carbon(self, config):
        """A simple proxy for carbon cost if direct measurement fails."""
        layer_weights = {'GCN': 1.0, 'GAT': 1.5, 'SAGE': 1.2}
        norm_weights = {'batch': 0.2, 'layer': 0.1, 'none': 0.0}
        return (layer_weights[config['conv_type']] +
                config['hidden_dim'] / 128.0 +
                config['num_layers'] / 4.0 +
                norm_weights[config['norm_type']])

    def proxy_train_and_eval(self, config, max_epochs, acc_threshold, check_every=5, min_epochs=15):
        """Quickly trains a model for a few epochs to evaluate its potential."""
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = GNNModel(config, self.input_dim, self.output_dim).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        criterion = nn.CrossEntropyLoss()
        tracker = EmissionsTracker(
            project_name=f"gnn_nas_proxy_{self.dataset_name}",
            measure_power_secs=15,
            output_file=self.output_file
        )
        tracker.start()
        best_acc = 0.0
        for epoch in range(max_epochs):
            model.train()
            for batch in self.train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = criterion(out, batch.y)
                loss.backward()
                optimizer.step()
            if (epoch + 1) % check_every == 0 or epoch == max_epochs - 1:
                acc = self.simple_eval(model, device)
                best_acc = max(best_acc, acc)
                if epoch + 1 >= min_epochs and best_acc < acc_threshold:
                    break # Early stop if model is not learning
                if best_acc >= acc_threshold:
                    break # Early success
        emissions = tracker.stop()
        carbon = emissions["emissions"] if emissions and isinstance(emissions, dict) and "emissions" in emissions else (self.estimate_proxy_carbon(config) * 0.01)
        return {'accuracy': best_acc, 'carbon': carbon, 'model': model}

    def simple_eval(self, model, device):
        """Evaluates a model on the validation set."""
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in self.val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item()
                total += batch.y.size(0)
        return correct / total if total > 0 else 0.0

    def train_and_evaluate(self, config, epochs):
        """Trains a model for the full number of epochs and evaluates it."""
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = GNNModel(config, self.input_dim, self.output_dim).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        criterion = nn.CrossEntropyLoss()
        tracker = EmissionsTracker(
            project_name=f"gnn_nas_{self.dataset_name}",
            measure_power_secs=15,
            output_file=self.output_file
        )
        tracker.start()
        model.train()
        for epoch in range(epochs):
            for batch in self.train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = criterion(out, batch.y)
                loss.backward()
                optimizer.step()
        accuracy = self.simple_eval(model, device)
        emissions = tracker.stop()
        carbon = emissions["emissions"] if emissions and isinstance(emissions, dict) and "emissions" in emissions else (self.estimate_proxy_carbon(config) * 0.01)
        return {'accuracy': accuracy, 'carbon': carbon, 'model': model}

    def mutate_architecture(self, base_config):
        """Creates a new architecture by mutating a single component of a base config."""
        # Mapping from search space attribute to config dictionary key
        component_map = {
            'conv_types': 'conv_type',
            'pooling_types': 'pooling_type',
            'activation_types': 'activation',
            'norm_types': 'norm_type',
            'hidden_dims': 'hidden_dim',
            'num_layers': 'num_layers',
            'dropout_rates': 'dropout'
        }
        if base_config is None: return self.search_space.sample_architecture()
        new_config = copy.deepcopy(base_config)
        component_to_mutate = random.choice(list(component_map.keys()))
        config_key = component_map[component_to_mutate]
        current_value = new_config[config_key]
        options = getattr(self.search_space, component_to_mutate)
        possible_values = [v for v in options if v != current_value]
        if possible_values:
            new_config[config_key] = random.choice(possible_values)
        return new_config

    def run_search(self):
        """Executes the entire evolutionary NAS process."""
        print(f"Starting Carbon-Aware GNN NAS on {self.dataset_name}...")
        self.baseline_config = {
            'conv_type': 'GCN', 'pooling_type': 'mean', 'activation': 'relu',
            'norm_type': 'batch', 'hidden_dim': 64, 'num_layers': 2, 'dropout': 0.2
        }
        print("Training baseline...")
        baseline_result = self.train_and_evaluate(self.baseline_config, epochs=60)
        print(f"Baseline - Accuracy: {baseline_result['accuracy']:.4f}, Carbon: {baseline_result['carbon']:.6f}")
        max_proxy_epochs_first_gen = 60
        max_proxy_epochs_others = 30
        proxy_acc_threshold = 0.70
        full_training_epochs = 60
        final_retrain_epochs = 100
        for generation in range(self.generations):
            print(f"\nGeneration {generation + 1}/{self.generations}")
            current_population = []
            if generation == 0:
                print("  Sampling initial viable population...")
                attempts = 0
                max_attempts = 300
                while len(current_population) < self.population_size and attempts < max_attempts:
                    config = self.search_space.sample_architecture()
                    result = self.proxy_train_and_eval(config, max_proxy_epochs_first_gen, proxy_acc_threshold)
                    if result['accuracy'] >= proxy_acc_threshold:
                        current_population.append({'config': config, 'proxy_acc': result['accuracy']})
                        print(f"    Found candidate {len(current_population)} with Proxy Acc={result['accuracy']:.4f}")
                    attempts += 1
                if len(current_population) < self.population_size:
                    print(f"  Warning: Only {len(current_population)} viable candidates found after {attempts} attempts!")
            else:
                print("  Mutating best architecture to create new population...")
                while len(current_population) < self.population_size:
                    config = self.mutate_architecture(self.best_architecture)
                    result = self.proxy_train_and_eval(config, max_proxy_epochs_others, proxy_acc_threshold)
                    if result['accuracy'] >= proxy_acc_threshold:
                        current_population.append({'config': config, 'proxy_acc': result['accuracy']})
                        print(f"    Found mutated candidate {len(current_population)} with Proxy Acc={result['accuracy']:.4f}")
            evaluated_population = []
            for candidate in current_population:
                full_result = self.train_and_evaluate(candidate['config'], epochs=full_training_epochs)
                reuse_score = self.calculate_block_reuse(candidate['config'], self.baseline_config)
                evaluated_population.append({
                    'config': candidate['config'],
                    'accuracy': full_result['accuracy'],
                    'carbon': full_result['carbon'],
                    'reuse': reuse_score,
                    'model': full_result['model']
                })
                print(f"    Candidate: Acc={full_result['accuracy']:.4f}, Carbon={full_result['carbon']:.6f}, Reuse={reuse_score:.3f}")
            if evaluated_population:
                for cand in evaluated_population:
                    cand['score'] = (cand['accuracy'] * 0.6
                                    + (1 - min(cand['carbon'], 0.1) / 0.1) * 0.2
                                    + cand['reuse'] * 0.2)
                evaluated_population.sort(key=lambda x: x['score'], reverse=True)
                best_of_gen = evaluated_population[0]
                if best_of_gen['score'] > self.best_score:
                    self.best_score = best_of_gen['score']
                    self.best_architecture = best_of_gen['config']
                    self.best_model = best_of_gen['model']
                    print(f"  New best architecture found! Score: {self.best_score:.4f}")
        if not self.best_architecture:
            print("No suitable architecture found. Exiting.")
            return
        print("\nRetraining best architecture on full data...")
        final_result = self.train_and_evaluate(self.best_architecture, epochs=final_retrain_epochs)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.best_model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in self.test_loader:
                batch = batch.to(device)
                out = final_result['model'](batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item()
                total += batch.y.size(0)
        test_accuracy = correct / total if total > 0 else 0.0
        print("\n--- Final Results ---")
        print(f"Best Architecture: {self.best_architecture}")
        print(f"Final Test Accuracy: {test_accuracy:.4f}")
        print(f"Final Carbon Footprint: {final_result['carbon']:.6f} kg CO2")
        print(f"Block Reuse Score: {self.calculate_block_reuse(self.best_architecture, self.baseline_config):.3f}")
        results_df = pd.DataFrame([{
            'dataset': self.dataset_name,
            'best_config': str(self.best_architecture),
            'test_accuracy': test_accuracy,
            'carbon_footprint': final_result['carbon'],
            'reuse_score': self.calculate_block_reuse(self.best_architecture, self.baseline_config)
        }])
        results_df.to_csv(f'{self.dataset_name.lower()}_gnn_nas_results.csv', index=False)

if __name__ == "__main__":
    nas_runner = CarbonAwareGNNNAS(dataset_name='NCI1')
    nas_runner.run_search()


Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 81.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 99.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 76.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.2 MB/s eta 0:00:00


Processing...
Done!
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Dataset: NCI1 | Graphs: 4110 | Classes: 2 | Features: 37
Starting Carbon-Aware GNN NAS on NCI1...
Training baseline...


[codecarbon WARNING @ 11:11:35] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 11:11:36] [setup] RAM Tracking...
[codecarbon INFO @ 11:11:36] [setup] CPU Tracking...
[codecarbon WARNING @ 11:11:37] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 11:11:37] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 11:11:37] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 11:11:37] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:11:37] [setup] GPU Tracking...
[codecarbon INFO @ 11:11:37] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:11:37] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation mode

Baseline - Accuracy: 0.6448, Carbon: 0.022000

Generation 1/5
  Sampling initial viable population...


...

    Candidate: Acc=0.6910, Carbon=0.034500, Reuse=0.714


[codecarbon WARNING @ 12:44:39] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 12:44:39] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 12:44:39] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 12:44:39] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 12:44:39] [setup] GPU Tracking...
[codecarbon INFO @ 12:44:39] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 12:44:39] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 12:44:39] >>> Tracker's metadata:
[codecarbon INFO @ 12:44:39]   Platform sys

    Candidate: Acc=0.7664, Carbon=0.036500, Reuse=0.571

Retraining best architecture on full data...


[codecarbon WARNING @ 12:45:03] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 12:45:03] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 12:45:03] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 12:45:03] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 12:45:03] [setup] GPU Tracking...
[codecarbon INFO @ 12:45:03] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 12:45:03] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 12:45:03] >>> Tracker's metadata:
[codecarbon INFO @ 12:45:03]   Platform sys


--- Final Results ---
Best Architecture: {'conv_type': 'SAGE', 'pooling_type': 'mean', 'activation': 'relu', 'norm_type': 'batch', 'hidden_dim': 64, 'num_layers': 5, 'dropout': 0.2}
Final Test Accuracy: 0.8054
Final Carbon Footprint: 0.031500 kg CO2
Block Reuse Score: 0.714
